In [1]:
# This notebook demonstrates a use case 
# of modelling the Galactic disc density using Red Clump giants
# while accounting for the Selection function

In [10]:
print('test')

import os, sys
loc = os.getcwd()
sys.path.insert(0,loc)	 
from rcdemo.packages_to_import import *
import rcdemo.dtools as dtools


1234


In [12]:
loc

'/Users/shouryapro/Documents/pdoc_work/py_scripts'